https://github.com/huggingface/transformers/tree/master/examples/seq2seq

In [43]:
%reload_ext pymarlin.core

# Data

CNN Daily mail

Download from https://cdn-datasets.huggingface.co/summarization/cnn_dm_v2.tgz -UseBasicParsing

In [1]:
# # !powershell wget https://cdn-datasets.huggingface.co/summarization/cnn_dm_v2.tgz -UseBasicParsing

# !powershell dir 'C:/Users/krkusuk/Downloads/'

# !powershell tar -xzvf 'C:/Users/krkusuk/Downloads/cnn_dm_v2.tgz'  # empty lines removed

# !powershell mv 'cnn_cln' 'D:/data'

# !powershell dir 'D:/data/cnn_cln'

In [2]:
import pandas as pd
with open('D:/data/cnn_cln/val.source', 'r', encoding = 'UTF-8') as f: 
    source = f.readlines()
source[0]

'(CNN)The only thing crazier than a guy in snowbound Massachusetts boxing up the powdery white stuff and offering it for sale online? People are actually buying it. For $89, self-styled entrepreneur Kyle Waring will ship you 6 pounds of Boston-area snow in an insulated Styrofoam box -- enough for 10 to 15 snowballs, he says. But not if you live in New England or surrounding states. "We will not ship snow to any states in the northeast!" says Waring\'s website, ShipSnowYo.com. "We\'re in the business of expunging snow!" His website and social media accounts claim to have filled more than 133 orders for snow -- more than 30 on Tuesday alone, his busiest day yet. With more than 45 total inches, Boston has set a record this winter for the snowiest month in its history. Most residents see the huge piles of snow choking their yards and sidewalks as a nuisance, but Waring saw an opportunity. According to Boston.com, it all started a few weeks ago, when Waring and his wife were shoveling deep 

In [3]:
with open('D:/data/cnn_cln/val.target', 'r', encoding = 'UTF-8') as f: 
    target = f.readlines()
target[0]

'A man in suburban Boston is selling snow online to customers in warmer states . For $89, he will ship 6 pounds of snow in an insulated Styrofoam box .\n'

In [4]:
len(source), len(target)

(13368, 13368)

In [15]:
import torch
import marlin
from pymarlin.core import *
from pymarlin.core.data_module import DataModule

## Data prepeocessing

In [16]:
def get_source_target(root = 'D:/data/cnn_cln', stage = 'val'):

    source = f'{root}/{stage}.source'
    target = f'{root}/{stage}.target'
    return source, target


In [17]:
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
class AnalyzeProcessor(data_module.DataProcessor):
    def __init__(self, source, target):
        with open(source, 'r', encoding = 'UTF-8') as f: 
            self.source = f.readlines()
        with open(target, 'r', encoding = 'UTF-8') as f: 
            self.target = f.readlines()
    def process(self):
        pass
    def analyze(self):
        self.df = pd.DataFrame({'source':self.source, 'target': self.target})
        display(self.df.head())
        print('Word length analysis:')
        wordlengths = processor.df.applymap(lambda x :  len(x.split()))
        print(wordlengths.describe())
        plt.plot(wordlengths)
        plt.legend(['source','target'])

        

In [23]:
# processor = AnalyzeProcessor(*get_source_target())
# processor.process()

# processor.analyze()

# DataModule

In [24]:
class SummarizationDataset(torch.utils.data.Dataset):
    def __init__(self, source, target):
        with open(source, 'r', encoding = 'UTF-8') as f: 
            self.source = f.readlines()
        with open(target, 'r', encoding = 'UTF-8') as f: 
            self.target = f.readlines()
    def __getitem__(self, i):
        return self.source[i].strip(), self.target[i].strip()
    def __len__(self):
        return len(self.target)
        

In [25]:
import marlin
class SummarizationDataModule(pymarlin.core.data_module.DataModule):
    '''
    Class which expects input data to have different files for source and target. 
    Returns dataset which returns non tokenized source and target text.
    '''
    def __init__(self):
        pass
    def prepare_ds(self, root= 'D:/data/cnn_cln'):
        self.root = root
        self.train_ds = SummarizationDataset(*get_source_target(root, 'train'))
        self.val_ds = SummarizationDataset(*get_source_target(root, 'val'))
    def get_train_dataset(self, *args, **kwargs):
        return self.train_ds

    def get_val_dataset(self, *args, **kwargs):
        return self.val_ds
    def get_test_dataset(self, *args, **kwargs):
        pass

In [26]:
dm= SummarizationDataModule()
dm.process_data(AnalyzeProcessor(*get_source_target(stage='val')))

,source,target
0,(CNN)The only thing crazier than a guy in snow...,A man in suburban Boston is selling snow onlin...
1,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend . Leag...
2,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...
3,(CNN)My vote for Father of the Year goes to Cu...,Ruben Navarrette: Schilling deserves praise fo...
4,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...


Word length analysis:
             source        target
count  13368.000000  13368.000000
mean     676.026406     57.910084
std      343.557667     25.613557
min       41.000000     10.000000
25%      413.750000     41.000000
50%      608.000000     54.000000
75%      868.000000     69.000000
max     1917.000000   1440.000000


(None, None)

The mean length is more than what Bart can accomodate. We will truncate for now and come up with more ideas later.

Feedback:
1. __init__() missing 2 required positional arguments: 'config' and 'logger' - unnecesary error. Don't know what to pass

In [27]:
dm.prepare_ds()
ds = dm.get_train_dataset()
len(ds),ds[0]

(287112,
 ('Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less lik

# TrainModule

In [31]:
# pymarlin.core.train_module.TrainModule??
from pymarlin.core import train_module

In [50]:
from transformers import BartForConditionalGeneration, BartTokenizerFast
from torch.utils.data import DataLoader
#too long import 
from pymarlin.utils.stats.basic_stats import BasicStats
class SummarizationBartTrainModule(train_module.TrainModule):
    def __init__(self, data:SummarizationDataModule , max_length = 128, max_lr = 2E-5):
        super().__init__()
        self.model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
        self.max_lr = max_lr
        self.max_length = max_length
        self.data = data
        self.stats = BasicStats.get_stats()
    def get_optimizers_schedulers(
        self, estimated_global_steps_per_epoch: int
        ):
        optimizer = torch.optim.Adam(self.model.parameters(),self.max_lr)
        return [optimizer],[]
    def get_train_dataloader(
        self, sampler:torch.utils.data.Sampler, batch_size:int
        ):
        train_ds = self.data.get_train_dataset()
        dl = DataLoader(train_ds, batch_size = batch_size, 
                        collate_fn= self.collate_fun, 
                        sampler = sampler(train_ds))
        return dl
    def get_val_dataloaders(
        self, sampler:torch.utils.data.Sampler, batch_size : int
    ):
        val_ds = self.data.get_val_dataset()
        dl = DataLoader(val_ds, batch_size = batch_size, 
                        collate_fn= self.collate_fun, 
                        sampler = sampler(val_ds))
        return dl
    def collate_fun(self,batch):
        source, target = torch.utils.data._utils.collate.default_collate(batch)
        #this is probably truncating. repeat positional embeddings and increase embedding layer size.
        source_tokens = self.tokenizer(
            list(source),
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length = self.max_length
        )
        target_tokens = self.tokenizer(
            list(target),
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length = self.max_length
        )
        labels = target_tokens["input_ids"]
        labels[labels[:, :] == self.model.config.pad_token_id] = -100
        source_tokens["labels"] =labels
        return source_tokens
    def train_step(
        self, global_step: int, batch, device
        ):
        batch = batch.to(device)
        result = self.model.forward(**batch)
        self.stats.update('train_loss', result.loss.item())
        return result.loss
    def val_step(self, global_step: int, batch, device):
        batch = batch.to(device)
        result = self.model.forward(**batch)
        return result.loss
        

In [51]:
tm = SummarizationBartTrainModule(dm)

# Trainer and Train manager

In [44]:
from pymarlin.core import trainer, train_manager
trainer = trainer.SingleProcess()

In [45]:
tmArgs = train_manager.TrainManagerArgs()
tmArgs.train_batch_size = 1
tmArgs.val_batch_size = 1
tmArgs.max_train_steps_per_epoch = 5
tmArgs.max_val_steps_per_epoch = 5

In [46]:
manager = train_manager.TrainManager(
    trainer = trainer,
    module= tm,
    args = tmArgs
)

SystemLog: 2021-01-15 16:20:26,733:WARNING : pymarlin.utils.writer.aml : 14 : AML writer failed to initialize.
SystemLog: 2021-01-15 16:20:26,733:INFO : pymarlin.utils.writer.aml : 15 : run =  None
SystemLog: 2021-01-15 16:20:26,741:INFO : pymarlin.utils.writer.tensorboard : 37 : Cleared directory logs (skipping azureml dirs)
SystemLog: 2021-01-15 16:20:26,746:INFO : pymarlin.utils.writer.tensorboard : 40 : Created tensorboard folder logs : []


In [47]:
manager.train()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/287112 [00:00<?, ?batch/s]

Validating


  0%|          | 0/13368 [00:00<?, ?it/s]

Finished training .. 
